по атакам
- завести код из репозитория TextAttack
- выбрать датасеты (хотя бы по одному) для русского и английского
- попробовать 1-2 атаки на этих данных, сравнить результаты
- проанализировать, насколько выбранные атаки подходят для работы с русским языком
- (бонус) предложить модификации атак для русского языка


In [ ]:
1) Загрузить датасет на классификацию
2) Прогнать через модель - оценить качество
3) Применить атаку на датасет 
4) Прогнать модель - сравнить результатф
5) попробовать 1-2 атаки для русского и английского

In [2]:
1+1

2

In [9]:
#!g1.1
import transformers
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")

In [10]:
#!g1.1
import textattack
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [11]:
#!g1.1
tokenizer

PreTrainedTokenizerFast(name_or_path='textattack/bert-base-uncased-imdb', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [12]:
#!g1.1
dataset = textattack.datasets.HuggingFaceDataset("imdb", split="test")
attack = textattack.attack_recipes.TextFoolerJin2019.build(model_wrapper)
# Attack 20 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(num_examples=20, log_to_csv="log.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()


Reusing dataset imdb (/tmp/xdg_cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
textattack: Loading datasets dataset imdb, split test.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log.csv
  0%|          | 0/20 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:  25%|██▌       | 5/20 [00:51<02:34, 10.32s/it]textattack: Saving checkpoint under "checkpoints/1638458804255.ta.chkpt" at 2021-12-02 15:26:44 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 0 / 0 / 10:  50%|█████     | 10/20 [05:17<05:17, 31.70s/it]textattack: Saving checkpoint under "checkpoints/1638459069679.ta.chkpt" at 2021-12-02 15:31:09 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 0 / 0 / 15:  75%|███████▌  | 15/20 [06:01<02:00, 24.11s/it]textattack: Saving checkpoint under "checkpoints/1638459114372.ta.chkpt" at 2021-12-02 15:31:54 after 15 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 0 / 1 / 19:  95%|█████████▌| 19/20 [06:29<00:20, 20.52s/it]





+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 19      |
| Number of failed attacks:     | 0       |
| Number of skipped attacks:    | 1       |
| Original accuracy:            | 95.0%   |
| Accuracy under attack:        | 0.0%    |
| Attack success rate:          | 100.0%  |
| Average perturbed word %:     | 7.22%   |
| Average num. words per input: | 273.5   |
| Avg num queries:              | 1055.58 |
+-------------------------------+---------+


[Succeeded / Failed / Skipped / Total] 19 / 0 / 1 / 20: 100%|██████████| 20/20 [06:32<00:00, 19.60s/it]textattack: Saving checkpoint under "checkpoints/1638459144715.ta.chkpt" at 2021-12-02 15:32:24 after 20 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 0 / 1 / 20: 100%|██████████| 20/20 [06:32<00:00, 19.60s/it]


In [15]:
#!g1.1

dataset = textattack.datasets.HuggingFaceDataset("imdb", split="test")
attack = textattack.attack_recipes.InputReductionFeng2018.build(model_wrapper)
# Attack 20 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(num_examples=20, log_to_csv="log2.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Reusing dataset imdb (/tmp/xdg_cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
textattack: Loading datasets dataset imdb, split test.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.
textattack: Logging to CSV at path log2.csv
  0%|          | 0/20 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  InputReduction(
    (maximizable):  True
  )
  (transformation):  WordDeletion
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:  25%|██▌       | 5/20 [00:22<01:07,  4.48s/it]textattack: Saving checkpoint under "checkpoints/1638459994758.ta.chkpt" at 2021-12-02 15:46:34 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 0 / 0 / 10:  50%|█████     | 10/20 [01:11<01:11,  7.14s/it]textattack: Saving checkpoint under "checkpoints/1638460043731.ta.chkpt" at 2021-12-02 15:47:23 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 0 / 0 / 15:  75%|███████▌  | 15/20 [01:30<00:30,  6.02s/it]textattack: Saving checkpoint under "checkpoints/1638460062688.ta.chkpt" at 2021-12-02 15:47:42 after 15 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 0 / 1 / 19:  95%|█████████▌| 19/20 [02:00<00:06,  6.33s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 95.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 37.83% |
| Average num. words per input: | 273.5  |
| Avg num queries:              | 442.79 |
+-------------------------------+--------+


[Succeeded / Failed / Skipped / Total] 19 / 0 / 1 / 20: 100%|██████████| 20/20 [02:03<00:00,  6.16s/it]textattack: Saving checkpoint under "checkpoints/1638460095557.ta.chkpt" at 2021-12-02 15:48:15 after 20 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 0 / 1 / 20: 100%|██████████| 20/20 [02:03<00:00,  6.16s/it]


In [16]:
#!g1.1

dataset = textattack.datasets.HuggingFaceDataset("imdb", split="test")
attack = textattack.attack_recipes.bae_garg_2019.BAEGarg2019.build(model_wrapper)
# Attack 20 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(num_examples=20, log_to_csv="log2.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Reusing dataset imdb (/tmp/xdg_cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
textattack: Loading datasets dataset imdb, split test.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log2.csv
  0%|          | 0/20 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 3 / 2 / 0 / 5:  25%|██▌       | 5/20 [01:24<04:14, 16.94s/it]textattack: Saving checkpoint under "checkpoints/1638460203268.ta.chkpt" at 2021-12-02 15:50:03 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 3 / 0 / 10:  50%|█████     | 10/20 [02:58<02:58, 17.89s/it]textattack: Saving checkpoint under "checkpoints/1638460297523.ta.chkpt" at 2021-12-02 15:51:37 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 6 / 0 / 15:  75%|███████▌  | 15/20 [04:34<01:31, 18.31s/it]textattack: Saving checkpoint under "checkpoints/1638460393193.ta.chkpt" at 2021-12-02 15:53:13 after 15 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 6 / 1 / 19:  95%|█████████▌| 19/20 [05:03<00:15, 15.98s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 95.0%  |
| Accuracy under attack:        | 30.0%  |
| Attack success rate:          | 68.42% |
| Average perturbed word %:     | 3.95%  |
| Average num. words per input: | 273.5  |
| Avg num queries:              | 488.42 |
+-------------------------------+--------+


[Succeeded / Failed / Skipped / Total] 13 / 6 / 1 / 20: 100%|██████████| 20/20 [05:05<00:00, 15.28s/it]textattack: Saving checkpoint under "checkpoints/1638460424196.ta.chkpt" at 2021-12-02 15:53:44 after 20 attacks.
[Succeeded / Failed / Skipped / Total] 13 / 6 / 1 / 20: 100%|██████████| 20/20 [05:05<00:00, 15.28s/it]


#!g1.1

dataset = textattack.datasets.HuggingFaceDataset("glue", split="test")
attack = textattack.attack_recipes.bae_garg_2019.BAEGarg2019.build(model_wrapper)
# Attack 20 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(num_examples=20, log_to_csv="log2.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

In [ ]:
#!g1.1
'glue', 'mrpc',

## завести модели на русском

In [ ]:
#!g1.1
dataset = textattack.datasets.HuggingFaceDataset("russian_super_glue", split="test")
attack = textattack.attack_recipes.TextFoolerJin2019.build(model_wrapper)
# Attack 20 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(num_examples=20, log_to_csv="log2.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()